In [3]:
import torch
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.optim import Adam
import torch.nn as nn

Preprocess

In [ ]:
transformer = transforms.Compose([]
    
)

Dataloader

In [5]:
trainPath=""

In [1]:
train = DataLoader(ImageFolder(trainPath, transformer), shuffle=True)

NameError: name 'DataLoader' is not defined

Network

Train

Eval